![diagram](Diagram.PNG)

## Plugin definition in common

### Manifest

In [1]:
# Manifest example
from manifests import BruteForceExploiterManifest
from config_fields import NumericField, ListField

manifest = BruteForceExploiterManifest(title="RDP Exploiter",
                                       plugin_name="rdp_exploiter",  # Could be automatically created somehow?
                                       description="Exploits RDP connections by brute-forcing. Warning! it's slow.",
                                       link_to_docs="www.beef.face",
                                       is_safe=True,
                                       # Right now no exploiter needs these. Implementing the ability to add options would require a refactoring of plugin selectors to also trigger the view of these
                                       # options
                                       options={
                                           "scan_timeout": NumericField(description="How much time to wait between each brute-force attempt. Increase to make the exploitation process quieter",
                                                                        title="RDP timeout"),
                                            "rdp_versions": ListField(all_values=["1.*", "2.*", "3.*"],
                                                                      title="Target versions",
                                                                      description="Which versions of RDP this plugin should target")
                                       })


### Config

In [2]:
# Config example

config = {
    "plugin_name": "rdp_exploiter",
    "enabled": True,
    # Don't validate these for now, as we only need the hard-coded smb-timeout
    "options":{
        "scan_timeout": 10,
        "rdp_versions": ["2.*", "3.*"]
    }
}

### Codebase

In [3]:
# Plugin codebase example (notice how it uses the options)
from time import sleep
from common.plugin_registry import register_plugin

def _scan_rdp_version(target):
    pass

def _attempt_brute_force(target, username, password):
    pass

def main(options):
    if not _scan_rdp_version(options['target']['ip']) in options["rdp_versions"]:
        raise RuntimeError(f"Target {options['target']} is running unsupported RDP version")
    for username, password in options['credentials']:
        if _attempt_brute_force(target, username, password):
            # Send exploitation event
            return
        sleep(options["scan_timeout"])

# IDK how we're going to register it, it's just a mock
register_plugin(manifest=manifest, config=config, entrypoint=main)

## Common actions

## Backend

### Loading the plugin

1. Load the plugin (make sure no other plugin with the same name already exists, validate the configuration against manifest)
2. Store configuration in the repository

### Resetting to default

1. Rebuild the configuration with plugin definitions

### Setting configuration

1. Validate the configuration against manifest.
2. Store the configuration in repository.

## UI

### Loading configuration page
1. Big changes as we need dedicated component. Take a look at the prototype

### Exporting configuration

No changes

### Importing configuration

Standard procedure, but when it comes to plugins it would be best to import as much as we can. So if we have a config and have imported a new plugin, that config won't go to waste. This is not priority 0 though

### Submitting configuration

Validation (see other sections)

### Storing the configuration

Validate the config against manifests

## Requirements

Agent needs to know the current config

Island needs to know default and current config

Interfaces need to know current, all available options and descriptions

### Single config vs separate configurations

**Single config**:
 - Less traffic
 - Would need to merge plugin configurations into it
 - Configurations are getting coupled

Config per plugin:
 - Easier loading
 - Harder resetting and import/export
 - More changes, refactoring repository

### YAML vs JSON vs XML

XML is not readable nor a standard in python. YAML is more readable, but JSON is probably the better option. The plugins will be developed in python most of the time. This means that
it's easier to dump the dictionary you tested with to json (yaml would require additional dependency). Validating YAML with jsonschema is also unorthodox, as to write a plugin you'll have to switch between the two notations. Lastly, YAML to JSONSchema would sacrifice some JSONSchema features, such as refs.

There's also [JDT](https://www.rfc-editor.org/rfc/rfc8927), but it seems to still be in "experimental" stage and it looks like a simplified version of jsonschema.

# Agent

## Running the plugins

Agent needs to run the entrypoint of the plugin and pass the relevant options (different to each plugin).


# UI prototype

# Usage

There are no good packages to display YAML as a form (only one I found: https://github.com/MaximeGoyette/react-yaml-form)

`react-jsonschema-form` takes in jsonschema and has everything we need, except UI component versatility.

We should also consider migrating our code to the official `react-jsonschema-form` package. Our current package `react-jsonschema-form-bs4` is 3 years old and unmaintained.

# Validation

`react-jsonschema-form` uses [ajv-validator](https://github.com/ajv-validator/ajv) which is fast and supports everything we need (including [formats](https://react-jsonschema-form.readthedocs.io/en/latest/usage/validation/#customformats)).

By default jsonschema supports these formats (in addition to the simple types):
Dates, Times, and Duration
Email Addresses
Hostnames
IP Addresses
Resource Identifiers
uri-template
JSON Pointers
regex

Corresponding `react-jsonschema-form` formats can be found [here](https://ajv.js.org/guide/formats.html)

Developer can specify the format in schema and if the interface understands it configuration will be validated.
However, if the interface is not familiar with the format, the format can't be defined in the schema.

## Limitations

We can't do references, conditional validation because plugins define a subschema, but references and conditions need to be defined at root level. We could add them separatelly, but that's a lot of work for a feature we probably don't need.

[Here](https://rjsf-team.github.io/react-jsonschema-form/) are some examples of what's possible (keep in mind the limitations above)

# GUI

For GUI we need a scalable component to select the plugin (like a dropdown, scrollable menu, etc) and an option window.

You can see some examples in the "gui" folder.

The mock we agreed upon is in "PluginOptionMock.png".
